# Trading Strategy Simulation
In this notebook, we will implement a simple trading strategy and simulate the trading process using historical limit order book (LOB) data. We will start by loading the LOB data and performing some feature engineering to create trading signals. 

The dataset used in this notebook is a resampled version of the LOB data, which contains 1-minute aggregated data. The data includes the following columns:
- **Datetime**: The timestamp of the data.
- **Min Ask Price**: The minimum ask price in one minute.
- **Max Bid Price**: The maximum bid price in one minute.
- **Total Ask Quantity sum**: The total ask quantity in one minute.
- **Total Bid Quantity sum**: The total bid quantity in one minute.

## Load Data
Use the resampled lob(1min) data to build a model and simulator for trading.

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from Ryan.packages.Trading_metrics import calculate_sharpe_ratio

In [32]:
lob_data=pd.read_csv('../datasets/resampled_lob_min.csv')
lob_data['Datetime'] = pd.to_datetime(lob_data['Datetime'])
lob_data

,Datetime,Min Ask Price,Max Bid Price,Total Bid Quantity sum,Total Ask Quantity sum
0,2025-01-02 00:00:00,271.841538,260.186154,22881.0,16799.0
1,2025-01-02 00:01:00,262.803138,258.206847,24821.0,12978.0
2,2025-01-02 00:02:00,267.596615,262.929478,33966.0,11786.0
3,2025-01-02 00:03:00,267.066274,262.751105,30396.0,15613.0
4,2025-01-02 00:04:00,268.358289,263.077540,28408.0,16458.0
...,...,...,...,...,...
62853,2025-07-01 08:13:00,151.230132,96.942053,18271.0,10399.0
62854,2025-07-01 08:14:00,200.771852,107.148148,16849.0,10211.0
62855,2025-07-01 08:15:00,174.894172,107.539877,29053.0,9511.0
62856,2025-07-01 08:16:00,174.897866,108.716463,33890.0,9471.0


## Feature Engineering
Create features like price change, bid-ask pressure difference, moving averages etc.

In [33]:
# 计算价格变动 Price Change
lob_data['Ask Price Change'] = lob_data['Min Ask Price'].diff()
lob_data['Bid Price Change'] = lob_data['Max Bid Price'].diff()

# 计算买卖压力差 Bid-Ask Pressure Difference
lob_data['Pressure Difference'] = lob_data['Total Bid Quantity sum'] - lob_data['Total Ask Quantity sum']

# 计算移动平均线 Moving Averages; Long and Short Windows
short_window = 5
long_window = 10

lob_data['Short Moving Avg Ask'] = lob_data['Min Ask Price'].rolling(window=short_window).mean()
lob_data['Long Moving Avg Ask'] = lob_data['Min Ask Price'].rolling(window=long_window).mean()
lob_data['Short Moving Avg Bid'] = lob_data['Max Bid Price'].rolling(window=short_window).mean()
lob_data['Long Moving Avg Bid'] = lob_data['Max Bid Price'].rolling(window=long_window).mean()

# 显示包含新特征的数据头几行
lob_data.head(15)

,Datetime,Min Ask Price,Max Bid Price,Total Bid Quantity sum,Total Ask Quantity sum,Ask Price Change,Bid Price Change,Pressure Difference,Short Moving Avg Ask,Long Moving Avg Ask,Short Moving Avg Bid,Long Moving Avg Bid
0,2025-01-02 00:00:00,271.841538,260.186154,22881.0,16799.0,NaN,NaN,6082.0,NaN,NaN,NaN,NaN
1,2025-01-02 00:01:00,262.803138,258.206847,24821.0,12978.0,-9.038400,-1.979306,11843.0,NaN,NaN,NaN,NaN
2,2025-01-02 00:02:00,267.596615,262.929478,33966.0,11786.0,4.793477,4.722631,22180.0,NaN,NaN,NaN,NaN
3,2025-01-02 00:03:00,267.066274,262.751105,30396.0,15613.0,-0.530341,-0.178374,14783.0,NaN,NaN,NaN,NaN
4,2025-01-02 00:04:00,268.358289,263.077540,28408.0,16458.0,1.292015,0.326436,11950.0,267.533171,NaN,261.430225,NaN
5,2025-01-02 00:05:00,265.378682,260.562412,28895.0,12759.0,-2.979607,-2.515128,16136.0,266.240600,NaN,261.505477,NaN
6,2025-01-02 00:06:00,266.854972,256.726519,34108.0,15650.0,1.476291,-3.835893,18458.0,267.050966,NaN,261.209411,NaN
7,2025-01-02 00:07:00,265.266362,260.334855,22859.0,13753.0,-1.588610,3.608336,9106.0,266.584916,NaN,260.690486,NaN
8,2025-01-02 00:08:00,267.154976,260.825449,19946.0,10646.0,1.888613,0.490593,9300.0,266.602656,NaN,260.305355,NaN
9,2025-01-02 00:09:00,267.283837,262.383706,32991.0,12274.0,0.128862,1.558257,20717.0,266.387766,266.960468,260.166588,260.798407


## Simulate Trading
### 1. Simple Moving Average Crossover Strategy
The simple moving average crossover strategy is a simple and effective strategy for trading. It is based on the moving averages of the stock prices. The strategy is to buy when the short moving average crosses above the long moving average and sell when the short moving average crosses below the long moving average.

We will define a simple trading strategy and implement a trading simulator to evaluate the performance of this strategy. Our strategy will be based on the following assumptions:
- **Buy Signal**: When the short-term moving average crosses above the long-term moving average, this may indicate an upward price trend, and we will take this as a buy signal.
- **Sell Signal**: When the short-term moving average crosses below the long-term moving average, this may indicate a downward price trend, and we will take this as a sell signal.
- **Position Limit**: To simplify the simulation, we assume that at any time we hold at most one unit of the asset.
- **Transaction Cost**: Each trade (buy or sell) will incur a certain percentage of transaction cost, here we assume it to be 0.1%.

### Implement Trading Simulator
1. **Initialization**: Set initial capital, record trading and position status.
2. **Signal Generation**: Generate trading signals based on the above buy and sell signal logic.
3. **Execute Trades**: Buy or sell based on the trading signals, and consider transaction costs.
4. **Performance Evaluation**: Calculate the final capital and return, as well as other possible performance metrics (e.g., maximum drawdown, Sharpe ratio, etc.).

接下来，我们将定义一个简单的交易策略，并实施一个交易模拟器来评估这个策略的表现。我们的策略将基于以下几个假设：

1. **买入信号**：当短期移动平均线从下穿越长期移动平均线时，这可能表明价格趋势向上，我们将这视为一个买入信号。
2. **卖出信号**：当短期移动平均线从上穿越长期移动平均线时，这可能表明价格趋势向下，我们将这视为一个卖出信号。
3. **持仓限制**：为简化模拟，我们假设在任何时候最多只持有一单位资产。
4. **交易成本**：每次交易（买入或卖出）将扣除一定比例的交易成本，这里我们假设为0.1%。

### 实施交易模拟器

1. **初始化**：设置初始资本，记录交易和持仓状态。
2. **信号生成**：根据上述买入和卖出信号的逻辑来生成交易信号。
3. **执行交易**：根据交易信号买入或卖出，并考虑交易成本。
4. **性能评估**：计算最终资本和收益率，以及其他可能的性能指标（如最大回撤、夏普比率等）。

In [27]:
# 初始化交易模拟器的参数
initial_capital = 100000.0  # 初始资本
capital = initial_capital  # 当前资本
position = 0  # 持仓数量，这里假设最多持有一单位资产
transaction_cost = 0.001  # 交易成本，0.1%

# 用于记录交易和资本变动的列表
trades = []
capital_history = [initial_capital]

# 遍历LOB数据，模拟交易逻辑
for i in range(1, len(lob_data)):
    # 买入信号：短期均线上穿长期均线，且当前无持仓
    if (lob_data['Short Moving Avg Ask'][i-1] < lob_data['Long Moving Avg Ask'][i-1]) and \
       (lob_data['Short Moving Avg Ask'][i] > lob_data['Long Moving Avg Ask'][i]) and \
       (position == 0):
        # 执行买入，考虑交易成本
        buy_price = lob_data['Min Ask Price'][i] * (1 + transaction_cost)
        capital -= buy_price  # 扣除买入成本
        position = 1  # 更新持仓状态
        trades.append(('BUY', lob_data['Datetime'][i], buy_price))
    
    # 卖出信号：短期均线下穿长期均线，且当前有持仓
    elif (lob_data['Short Moving Avg Ask'][i-1] > lob_data['Long Moving Avg Ask'][i-1]) and \
         (lob_data['Short Moving Avg Ask'][i] < lob_data['Long Moving Avg Ask'][i]) and \
         (position == 1):
        # 执行卖出，考虑交易成本
        sell_price = lob_data['Min Ask Price'][i] * (1 - transaction_cost)
        capital += sell_price  # 增加卖出收益
        position = 0  # 更新持仓状态
        trades.append(('SELL', lob_data['Datetime'][i], sell_price))
    
    # 更新资本历史记录
    capital_history.append(capital)

# 计算最终收益
final_return = (capital - initial_capital) / initial_capital
num_trades = len(trades)

# Sharp Ratio
# 将资本历史转换为日收益率
capital_history_array = np.array(capital_history)
daily_returns = np.diff(capital_history_array) / capital_history_array[:-1]
sharpe_ratio = calculate_sharpe_ratio(daily_returns)

# 输出优化后的交易次数和最终收益
print(f'Number of trades: {num_trades}')
print(f'Final return: {final_return:.2%}')
print(f'Final capital: {capital:.2f}')
print(f'Sharpe Ratio: {sharpe_ratio:.3f}') 

Number of trades: 14077
Final return: -121.79%
Final capital: -21789.33
Sharpe Ratio: -0.004


模拟交易的结果显示：

- 进行了总共14,077次交易。
- 最终收益率为-1217.89%，这表明按照这个简单的交易策略，初始资本损失了大部分。
- Sharpe Ratio为-0.004，这表明策略的风险调整收益率为负。夏普比率相对较低，这可能是由于日收益率的波动性较大或平均回报率较低所致。为了提高夏普比率，我们需要提高策略的平均回报率或降低回报率的波动性。

### 分析和下一步

这个简单的交易策略导致了巨大的亏损，这可能是由于以下几个原因：

1. **交易成本的影响**：在每次交易中都考虑了0.1%的交易成本，频繁的交易使得成本累积，对收益产生了显著的负面影响。
2. **信号生成逻辑**：使用移动平均线生成买卖信号的方法可能不够准确或适合这个特定的数据集。移动平均线是一种趋势跟踪指标，它可能会在价格波动时产生滞后的信号。
3. **市场噪音**：短期的市场波动可能导致频繁的买卖信号，而这些信号可能并不总是代表真正的市场趋势。

#### 下一步
首先尝试通过优化移动平均线的窗口大小来减少交易次数，然后引入冷却期来避免频繁交易。

In [28]:
# 调整移动平均线窗口大小
short_window_opt = 10  # 优化后的短期窗口
long_window_opt = 20  # 优化后的长期窗口

# 计算优化后的移动平均线
lob_data['Opt Short Moving Avg Ask'] = lob_data['Min Ask Price'].rolling(window=short_window_opt).mean()
lob_data['Opt Long Moving Avg Ask'] = lob_data['Min Ask Price'].rolling(window=long_window_opt).mean()

# 引入冷却期
cooldown_period = 5  # 冷却期，单位是分钟
cooldown_counter = 0  # 冷却计数器

# 重新初始化交易模拟器的参数
capital = initial_capital  # 重置当前资本
position = 0  # 重置持仓数量
trades = []  # 重置交易记录
capital_history = [initial_capital]  # 重置资本历史

# 优化后的交易模拟逻辑
for i in range(1, len(lob_data)):
    # 更新冷却计数器
    if cooldown_counter > 0:
        cooldown_counter -= 1
    
    # 买入信号：短期均线上穿长期均线，无持仓，冷却期结束
    if (lob_data['Opt Short Moving Avg Ask'][i-1] < lob_data['Opt Long Moving Avg Ask'][i-1]) and \
       (lob_data['Opt Short Moving Avg Ask'][i] > lob_data['Opt Long Moving Avg Ask'][i]) and \
       (position == 0) and (cooldown_counter == 0):
        # 执行买入
        buy_price = lob_data['Min Ask Price'][i] * (1 + transaction_cost)
        capital -= buy_price
        position = 1
        trades.append(('BUY', lob_data['Datetime'][i], buy_price))
        cooldown_counter = cooldown_period  # 重置冷却计数器
    
    # 卖出信号：短期均线下穿长期均线，有持仓，冷却期结束
    elif (lob_data['Opt Short Moving Avg Ask'][i-1] > lob_data['Opt Long Moving Avg Ask'][i-1]) and \
         (lob_data['Opt Short Moving Avg Ask'][i] < lob_data['Opt Long Moving Avg Ask'][i]) and \
         (position == 1) and (cooldown_counter == 0):
        # 执行卖出
        sell_price = lob_data['Min Ask Price'][i] * (1 - transaction_cost)
        capital += sell_price
        position = 0
        trades.append(('SELL', lob_data['Datetime'][i], sell_price))
        cooldown_counter = cooldown_period  # 重置冷却计数器
    
    capital_history.append(capital)

# 重新计算最终收益
final_return_opt = (capital - initial_capital) / initial_capital
num_trades_opt = len(trades)

# Sharp Ratio
# 将资本历史转换为日收益率
capital_history_array = np.array(capital_history)
daily_returns = np.diff(capital_history_array) / capital_history_array[:-1]
sharpe_ratio = calculate_sharpe_ratio(daily_returns)

# 输出优化后的交易次数和最终收益
print(f'Number of trades (optimized): {num_trades_opt}')
print(f'Final return (optimized): {final_return_opt:.2%}')
print(f'Final capital (optimized): {capital:.2f}')
print(f'Sharpe Ratio (optimized): {sharpe_ratio:.3f}')  

Number of trades (optimized): 4941
Final return (optimized): -47.52%
Final capital (optimized): 52476.66
Sharpe Ratio (optimized): -0.011


### 优化后的模拟交易结果显示：

- 进行了总共4,941次交易，相比之前的策略，交易次数有显著减少。
- 最终收益率为-475.23%，虽然仍然是亏损状态，但相比之前的策略，亏损幅度有所减少。
- Sharpe Ratio为-0.011，仍然是负值，但相比之前的策略有所提高。

### 分析和进一步的步骤
通过调整移动平均线的窗口大小并引入冷却期，我们成功减少了交易次数并降低了亏损幅度。然而，该策略仍然未能实现盈利。这可能是因为：

- 移动平均线策略本质上是趋势跟踪的，可能不适合所有类型的市场环境，尤其是在高波动或无明显趋势的市场中。
- 冷却期虽然减少了交易次数，但可能也导致错过了一些有利的交易机会。

### 为了进一步优化模型，我们可以考虑以下几个方向：

- 更复杂的交易信号：结合更多技术指标，使用机器学习模型来识别交易信号，可能能更准确地捕捉市场机会。
- 动态调整参数：根据市场条件动态调整策略参数，如移动平均线的窗口大小、冷却期长度等。
- 风险管理：引入更复杂的风险管理措施，如动态止损、止盈点，根据市场波动性调整仓位大小等。


### 2. RSI (Relative Strength Index) Strategy
为了进一步优化模型，我们可以考虑以下策略：

使用相对强弱指数（RSI）来生成交易信号。RSI是一种用于衡量价格波动的技术指标，它的取值范围在0到100之间。一般来说，RSI高于70可能表示价格过高，RSI低于30可能表示价格过低。基于这一特性，我们可以使用RSI来生成买卖信号。
- 买入：当RSI低于30且无持仓时，我们认为价格可能过低，可以考虑买入。
- 卖出：当RSI高于70且有持仓时，我们认为价格可能过高，可以考虑卖出。

### RSI Strategy Implementation
- Buy Signal: When the RSI is below 30 and there is no position.
- Sell Signal: When the RSI is above 70 and there is a position.

In [29]:
def calculate_rsi(data, window=14):
    """
    计算给定数据的相对强弱指数（RSI）。
    
    :param data: 包含价格数据的Pandas Series。
    :param window: 用于计算RSI的窗口大小，默认为14。
    :return: 包含RSI值的Pandas Series。
    """
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# 计算Min Ask Price的RSI
lob_data['RSI'] = calculate_rsi(lob_data['Min Ask Price'], window=14)

# 使用RSI改进交易信号
# 初始化参数
capital = initial_capital
position = 0
trades = []
capital_history = [initial_capital]

# 使用RSI的交易模拟逻辑
for i in range(1, len(lob_data)):
    # 买入信号：RSI低于30且无持仓
    if lob_data['RSI'][i] < 30 and position == 0:
        # 执行买入
        buy_price = lob_data['Min Ask Price'][i] * (1 + transaction_cost)
        capital -= buy_price
        position = 1
        trades.append(('BUY', lob_data['Datetime'][i], buy_price))
    
    # 卖出信号：RSI高于70且有持仓
    elif lob_data['RSI'][i] > 70 and position == 1:
        # 执行卖出
        sell_price = lob_data['Min Ask Price'][i] * (1 - transaction_cost)
        capital += sell_price
        position = 0
        trades.append(('SELL', lob_data['Datetime'][i], sell_price))
    
    capital_history.append(capital)

# 重新计算最终收益和夏普比率
final_return_rsi = (capital - initial_capital) / initial_capital
num_trades_rsi = len(trades)

# 将资本历史转换为日收益率
capital_history_array = np.array(capital_history)
daily_returns = np.diff(capital_history_array) / capital_history_array[:-1]
sharpe_ratio = calculate_sharpe_ratio(daily_returns)

# 输出RSI策略的交易次数和最终收益
print(f'Number of trades (RSI): {num_trades_rsi}')
print(f'Final return (RSI): {final_return_rsi:.2%}')
print(f'Final capital (RSI): {capital:.2f}')
print(f'Sharpe Ratio (RSI): {sharpe_ratio:.3f}')

Number of trades (RSI): 263
Final return (RSI): 5.67%
Final capital (RSI): 105673.04
Sharpe Ratio (RSI): 0.004


As the result shows that the RSI strategy has significantly reduced the number of trades and improved the final return and Sharpe ratio. The final return is 5.67% and the Sharpe ratio is 0.004. 